## Limpieza de base de datos de contagios
Cargando los datos

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np

df_original = pd.read_csv('https://cloud.minsa.gob.pe/s/Y8w3wHsEdYQSZRp/download',sep=";")
df_original.head()

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
0,20210302,7320cabdc1aaca6c59014cae76a134e6,LIMA REGION,HUAROCHIRI,SAN ANTONIO,PR,41.0,FEMENINO,20200526.0
1,20210302,e81602051997ace8340bb8c18fe24c65,APURIMAC,ABANCAY,ABANCAY,PR,32.0,FEMENINO,20200425.0
2,20210302,71ecb6bccb248b0bb2ac72ed51b5e979,APURIMAC,ABANCAY,ABANCAY,PR,34.0,FEMENINO,20200429.0
3,20210302,566af4276cbe9359abe93f9aa86396c3,APURIMAC,ANDAHUAYLAS,ANDAHUAYLAS,PR,40.0,FEMENINO,20200426.0
4,20210302,027561e9d126e7c283d79c02cede562d,APURIMAC,ABANCAY,ABANCAY,PR,40.0,FEMENINO,20200428.0



Dropeando y ordenado


In [2]:
# Generando una base a parte
df = df_original
# Drop a las observaciones sin fecha de resultado
df.dropna(subset=['FECHA_RESULTADO'], axis=0, inplace=True)
# Dando formato de fecha a la fecha de resultado de la prueba del covid
import datetime
df["FECHA_RESULTADO"] = df["FECHA_RESULTADO"].astype(int)
df["FECHA_RESULTADO"] = pd.to_datetime(df.FECHA_RESULTADO, format='%Y%m%d')
# Drop a las variables poco relevantes por el momento
df = df.drop(['FECHA_CORTE','EDAD','METODODX','SEXO'], axis=1)
# Reordenamiento de variables
df = df[['DEPARTAMENTO', 'PROVINCIA','DISTRITO','FECHA_RESULTADO','UUID']]
# Ordenando por Dep, Prov, Dist y Fecha
df = df.sort_values(by=['DEPARTAMENTO', 'PROVINCIA','DISTRITO','FECHA_RESULTADO'])
# Agrupando las cantidad de UUID = Personas contagiadas
df = df.groupby(by=['DEPARTAMENTO', 'PROVINCIA','DISTRITO','FECHA_RESULTADO'], as_index=False).count()
# Visualización del resultado
df.head(20)

,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA_RESULTADO,UUID
0,AMAZONAS,BAGUA,ARAMANGO,2020-04-07,1
1,AMAZONAS,BAGUA,ARAMANGO,2020-04-12,1
2,AMAZONAS,BAGUA,ARAMANGO,2020-04-17,1
3,AMAZONAS,BAGUA,ARAMANGO,2020-04-18,1
4,AMAZONAS,BAGUA,ARAMANGO,2020-04-20,1
5,AMAZONAS,BAGUA,ARAMANGO,2020-04-27,2
6,AMAZONAS,BAGUA,ARAMANGO,2020-04-30,1
7,AMAZONAS,BAGUA,ARAMANGO,2020-05-01,7
8,AMAZONAS,BAGUA,ARAMANGO,2020-05-02,1
9,AMAZONAS,BAGUA,ARAMANGO,2020-05-05,2


Formateando fecha para trabajar semanalmente


In [3]:
# Dando el formato de semana
df['SEMANA'] = df['FECHA_RESULTADO'].dt.strftime('%Y-%U')
# Dropeando la Fecha Larga
df = df.drop(['FECHA_RESULTADO'], axis=1)
# Agrupando por Dep, Prov, Dist, Semana
df = df.groupby(by=['DEPARTAMENTO', 'PROVINCIA','DISTRITO','SEMANA'], as_index=False).sum()
# Visualizando Base
df

,DEPARTAMENTO,PROVINCIA,DISTRITO,SEMANA,UUID
0,AMAZONAS,BAGUA,ARAMANGO,2020-14,1
1,AMAZONAS,BAGUA,ARAMANGO,2020-15,3
2,AMAZONAS,BAGUA,ARAMANGO,2020-16,1
3,AMAZONAS,BAGUA,ARAMANGO,2020-17,11
4,AMAZONAS,BAGUA,ARAMANGO,2020-18,2
...,...,...,...,...,...
40066,UCAYALI,PURUS,PURUS,2021-02,3
40067,UCAYALI,PURUS,PURUS,2021-03,2
40068,UCAYALI,PURUS,PURUS,2021-04,2
40069,UCAYALI,PURUS,PURUS,2021-06,5


Cargando ubigeos

In [4]:
# Cargando los datos
url = "https://raw.githubusercontent.com/vacunatepe/automatcoviddata/main/ubigeo.csv" 
ubi = pd.read_csv(url,dtype={'Ubigeo': str})

# Ordenando
ubi = ubi[['Departamento','Provincia','Distrito','Ubigeo']]
# Renombrando para combinar
ubi = ubi.rename(columns={'Departamento': 'DEPARTAMENTO','Provincia':'PROVINCIA','Distrito':'DISTRITO','Ubigeo':'UBIGEO'})
# Vistazo
ubi

,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,010101
1,AMAZONAS,CHACHAPOYAS,ASUNCION,010102
2,AMAZONAS,CHACHAPOYAS,BALSAS,010103
3,AMAZONAS,CHACHAPOYAS,CHETO,010104
4,AMAZONAS,CHACHAPOYAS,CHILIQUIN,010105
...,...,...,...,...
1869,UCAYALI,PADRE ABAD,IRAZOLA,250302
1870,UCAYALI,PADRE ABAD,CURIMANA,250303
1871,UCAYALI,PADRE ABAD,NESHUYA,250304
1872,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,250305


Asignado ubigeo a la base principal

In [5]:
# Combinando
new_df = pd.merge(df, ubi,  how='left', left_on=['DEPARTAMENTO', 'PROVINCIA','DISTRITO'], right_on = ['DEPARTAMENTO', 'PROVINCIA','DISTRITO'])
# Ordenando
new_df = new_df[['DEPARTAMENTO', 'PROVINCIA','DISTRITO','UBIGEO','SEMANA','UUID']]
# Vistazo
new_df

,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,SEMANA,UUID
0,AMAZONAS,BAGUA,ARAMANGO,010202,2020-14,1
1,AMAZONAS,BAGUA,ARAMANGO,010202,2020-15,3
2,AMAZONAS,BAGUA,ARAMANGO,010202,2020-16,1
3,AMAZONAS,BAGUA,ARAMANGO,010202,2020-17,11
4,AMAZONAS,BAGUA,ARAMANGO,010202,2020-18,2
...,...,...,...,...,...,...
40066,UCAYALI,PURUS,PURUS,250401,2021-02,3
40067,UCAYALI,PURUS,PURUS,250401,2021-03,2
40068,UCAYALI,PURUS,PURUS,250401,2021-04,2
40069,UCAYALI,PURUS,PURUS,250401,2021-06,5


Descargando

In [6]:
new_df.to_csv('contagiosUS.csv')